In [152]:
%logstop
%logstart -rtq ~/.logs/dw.py append
import seaborn as sns
sns.set()

In [3]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [ ]:
#!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

## Loading the data

Similar to the `pw` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a pandas `DataFrame` by making use of the `pd.read_csv` function to parse the data into a `DataFrame`. You may want to check the pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [`pw` miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [4]:
import pandas as pd
import numpy as np

In [5]:
# load the 2017 data
scripts = pd.read_csv("dw-data/201701scripts_sample.csv.gz")


In [6]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = pd.read_csv("dw-data/practices.csv.gz",names=col_names)


In [7]:
chem = pd.read_csv("dw-data/chem.csv.gz")


Now that we've loaded in the data, let's first replicate our results from the `pw` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `pw` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `pw` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: `[('act_cost', (total, mean, std, q25, median, q75)), ...]`

In [ ]:
scripts.describe()

In [ ]:
scripts.describe()["nic"]["50%"]

In [ ]:
results=scripts.describe().round(2)
colnames=["items","quantity","nic","act_cost"]
summary_stats=[]
for col in colnames:
    summary_stats.append( (col,(np.sum(scripts[col]),
                              results[col]["mean"], 
                              results[col]["std"], 
                              results[col]["25%"],
                              results[col]["50%"], 
                              results[col]["75%"])  ))

In [ ]:
summary_stats = [('items', (0,) * 6), ('quantity', (0,) * 6), ('nic', (0,) * 6), ('act_cost', (0,) * 6)]

In [ ]:
grader.score.dw__summary_statistics(summary_stats)

## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [ ]:
bnfgroup=scripts.groupby("bnf_name")

In [ ]:
max(bnfgroup["items"].sum())

In [ ]:
most_common_item = [(bnfgroup["items"].sum().idxmax(axis=1), max(bnfgroup["items"].sum()))]

In [ ]:
grader.score.dw__most_common_item(most_common_item)

## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the `pw` miniproject).

In [ ]:
practices.head()

In [ ]:
len(practices["code"].unique())

In [ ]:
len(practices["post_code"].unique())

In [ ]:
prac

In [ ]:
min(practices[practices["code"]=="A81003"].post_code)

In [ ]:
unique_codes=practices["code"].unique()

In [ ]:
ans_practices=pd.DataFrame(columns=practices.columns)

In [ ]:
ans_practices.shape

In [ ]:
ans_practices=ans_practices.drop_duplicates(subset=["code","post_code"])

In [ ]:
ans_practices.shape

In [ ]:
ans_practices.value_counts()

In [ ]:
for c in unique_codes:
    val=min(practices[practices["code"]==c].post_code)
    ans_practices=ans_practices.append(practices[(practices["code"]==c) & (practices["post_code"]==val)])
    
    
    

In [ ]:
len(ans_practices["post_code"].unique())

In [ ]:
practices.post_code

In [ ]:
scripts.head()

In [ ]:
data=scripts.merge(ans_practices,left_on="practice",right_on="code",how="left")

In [ ]:
data["post_code"].unique

In [ ]:
data.shape

In [ ]:
postgroup=data.groupby(["post_code","bnf_name"])

In [ ]:
newdata=postgroup["items"].sum()

In [ ]:
newdata.index[:2]

In [ ]:
newdata["B11 4BW"]

In [ ]:
mydata=pd.DataFrame(newdata)

In [ ]:
mydata.columns

In [ ]:
mydata.reset_index(inplace=True)

In [ ]:
mydata.columns

In [ ]:
unique_postcode=mydata['post_code'].unique()

In [ ]:
i=mydata[mydata['post_code']=='B11 4BW']['items'].idxmax()

In [ ]:
mydata.loc[i,'items']

In [ ]:
mydata.loc[[100,20,30]]['bnf_name'].sort_values().index[0]

In [ ]:
results=[]
for p in unique_postcode:
    total=mydata[mydata['post_code']==p]['items'].sum()
    m=mydata[mydata['post_code']==p]['items'].max()
    i=mydata[(mydata['items']==m) & (mydata["post_code"]==p)]["bnf_name"].sort_values().index[0]
    
    pre=list(mydata.loc[i])
    pre[2]=pre[2]/total
    res=tuple(pre)
    results.append(res)
    

In [ ]:
results=[]
for p in unique_postcode:
    total=mydata[mydata['post_code']==p]['items'].sum()
    i=mydata[mydata['post_code']==p]['items'].idxmax()

    pre=list(mydata.loc[i])
    pre[2]=pre[2]/total
    res=tuple(pre)
    results.append(res)

In [ ]:
results[:5]

In [ ]:
results[:5]

In [ ]:
results.sort()

In [ ]:
items_by_region = [("B11 4BW", "Salbutamol_Inha 100mcg (200 D) CFF", 0.0310589063)] * 100

In [ ]:
grader.score.dw__items_by_region(results[:100])

## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [8]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [9]:
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [12]:
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


In [15]:
pattern='|'.join([f'(?i){opi}' for opi in opioids]) 

In [17]:
pattern

'(?i)morphine|(?i)oxycodone|(?i)methadone|(?i)fentanyl|(?i)pethidine|(?i)buprenorphine|(?i)propoxyphene|(?i)codeine'

In [37]:
opi_codes=chem.loc[chem['NAME'].str.lower().str.contains(pattern, case=False)]['CHEM SUB'].tolist()

In [38]:
len(opi_codes)

35

In [25]:
scripts.shape

(973193, 7)

In [27]:
scripts["isopi"]=scripts['bnf_code'].isin(opi_codes)

In [29]:
scripts.shape

(973193, 8)

In [30]:
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,isopi
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,False
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,False
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,False
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,False
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,False


In [33]:
len(scripts.loc[scripts["isopi"]])

34843

In [35]:
scripts.groupby("practice")["isopi"].mean()

practice
A81005    0.033179
A81007    0.043329
A81011    0.046556
A81012    0.042793
A81017    0.038140
            ...   
Y05570    0.090909
Y05583    0.000000
Y05597    0.000000
Y05660    0.023055
Y05670    0.000000
Name: isopi, Length: 856, dtype: float64

In [51]:
scripts.dtypes

practice     object
bnf_code     object
bnf_name     object
items         int64
nic         float64
act_cost    float64
quantity      int64
isopi          bool
dtype: object

In [54]:
data["isopi"]=data["isopi"]==True

In [55]:
data.dtypes

practice     object
bnf_code     object
bnf_name     object
items         int64
nic         float64
act_cost    float64
quantity      int64
isopi          bool
CHEM SUB     object
NAME         object
dtype: object

In [10]:
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


In [39]:
data=scripts.merge(chem,how="left",left_on="bnf_code",right_on="CHEM SUB")

In [ ]:
data.head()

In [ ]:
data["isopi"]=data["NAME"].str.contains("morphine")

In [ ]:
data.head()

In [ ]:
data.isopi.value_counts()

In [ ]:
pattern='|'.join(opioids)
    

In [ ]:
pattern

In [ ]:
data["isopi"]=data["NAME"].str.contains(pattern)

In [ ]:
data.isopi.value_counts()

In [11]:
pattern='|'.join([f'(?i){opi}' for opi in opioids]) 

In [12]:
pattern

'(?i)morphine|(?i)oxycodone|(?i)methadone|(?i)fentanyl|(?i)pethidine|(?i)buprenorphine|(?i)propoxyphene|(?i)codeine'

In [46]:
data["isopi"]=data["NAME"].str.lower().str.contains(pattern,case=False)

In [17]:
data.isopi.value_counts()

False    820418
True      34843
Name: isopi, dtype: int64

In [47]:
data.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,isopi,CHEM SUB,NAME
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,False,0106020C0,Bisacodyl
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,False,0106040M0,Macrogol 3350
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,False,0301011R0,Salbutamol
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,False,0304010G0,Chlorphenamine Maleate
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,False,0401020K0,Diazepam


Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [56]:
data.groupby("practice")["isopi"].mean()

practice
A81005    0.033135
A81007    0.043269
A81011    0.046526
A81012    0.042761
A81017    0.038122
            ...   
Y05570    0.090909
Y05583    0.000000
Y05597    0.000000
Y05660    0.023055
Y05670    0.000000
Name: isopi, Length: 856, dtype: float64

In [43]:
practice_group['isopi'].mean()

DataError: No numeric types to aggregate

In [ ]:
scripts_per_practice=dict(practice_group.items.count())

In [ ]:
scripts_per_practice

In [ ]:
practice_group.isopi.value_counts()

In [ ]:
res=practice_group.isopi.value_counts()

In [ ]:
res=pd.DataFrame(res.reset_index(name="value_c"))

In [ ]:
res

In [ ]:
dict(zip(res.practice,res.value_c))

In [ ]:
res.loc[res["isopi"]==True,["practice","value_c"]]

In [ ]:
opioids_per_practice=dict(zip(res.practice,res.value_c))

In [ ]:
#{key:no_opioids_per_practice[key]/scripts_per_practice[key] for key in scripts_per_practice}

opioids_per_practice={}
for key in scripts_per_practice:
    if key in no_opioids_per_practice:
        opioids_per_practice[key]=no_opioids_per_practice[key]/scripts_per_practice[key]
    else:
        opioids_per_practice[key]=0

In [ ]:
#opioids_per_practice = dict(zip(res.practice,res.value_c))

In [57]:
opioids_per_practice=data.groupby("practice")["isopi"].mean()

In [59]:
opioids_per_practice["A81005"]

0.03313452617627568

In [60]:
relative_opioids_per_practice=opioids_per_practice-data["isopi"].mean()

In [62]:
relative_opioids_per_practice.head()

practice
A81005   -0.002638
A81007    0.007496
A81011    0.010754
A81012    0.006988
A81017    0.002349
Name: isopi, dtype: float64

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [ ]:
relative_opioids_per_practice={}
for key in scripts_per_practice:
    if key in opioids_per_practice:
        relative_opioids_per_practice[key]=scripts_per_practice[key]-opioids_per_practice[key]
    else:
        relative_opioids_per_practice[key]=scripts_per_practice[key]-0

In [ ]:
len(relative_opioids_per_practice)

In [ ]:
len(scripts_per_practice)

In [ ]:
len(opioids_per_practice)

In [ ]:
relative_opioids_per_practice

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [63]:
sum_opi_per_practice=data.groupby("practice")["isopi"].sum()

In [74]:
scripts_per_practice=data.groupby("practice")["bnf_code"].count()

In [65]:
sum_opi_per_practice.head()

practice
A81005    50
A81007    63
A81011    73
A81012    57
A81017    82
Name: isopi, dtype: int64

In [75]:
scripts_per_practice.head()

practice
A81005    1509
A81007    1456
A81011    1569
A81012    1333
A81017    2151
Name: bnf_code, dtype: int64

In [76]:
standard_error_per_practice=data["isopi"].std()/(scripts_per_practice**0.5)

In [77]:
standard_error_per_practice.head()

practice
A81005    0.004781
A81007    0.004867
A81011    0.004689
A81012    0.005087
A81017    0.004004
Name: bnf_code, dtype: float64

In [78]:
opioid_scores=relative_opioids_per_practice/standard_error_per_practice

In [79]:
opioid_scores.head()

practice
A81005   -0.551807
A81007    1.540186
A81011    2.293528
A81012    1.373723
A81017    0.586611
dtype: float64

In [ ]:
sd=np.std(list(relative_opioids_per_practice.values()))

In [ ]:
standard_error_per_practice={key:sd/np.sqrt(scripts_per_practice[key]) for key in scripts_per_practice}

In [ ]:
standard_error_per_practice

In [ ]:
opioid_scores={key:relative_opioids_per_practice[key]/standard_error_per_practice[key] for key in relative_opioids_per_practice}

In [ ]:
opioid_scores

In [ ]:
#standard_error_per_practice = ...
#opioid_scores = ...

In [72]:
opioid_scores['Y01852']

11.700971787589962

In [73]:
scripts_per_practice['Y01852']

NameError: name 'scripts_per_practice' is not defined

The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_code, practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [81]:
from heapq import nlargest

In [82]:
practices

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ
...,...,...,...,...,...,...,...
12015,Y05752,MALVERN FRAILTY UNIT,MALVERN COMM. HOSPITAL,185 WORCESTER ROAD,MALVERN,WORCESTERSHIRE,WR14 1EX
12016,Y05757,VC WEST LANCS HC-UCC,ORMSKIRK DG HOSPITAL,WIGAN ROAD,ORMSKIRK,LANCASHIRE,L39 2AZ
12017,Y05758,VC SKELMERSDALE UCC,CONCOURSE SHOPPING CENTRE,116-8 SOUTH WAY,SKELMERSDALE,LANCASHIRE,WN8 6LJ
12018,Y05762,WEST MERTON ACCESS HUB,THE NELSON HEALTH CENTRE,KINGSTON ROAD,WIMBLEDON,NaN,SW20 8DA


In [88]:
first_100=opioid_scores.sort_values(ascending=False).head(100)

In [109]:
first_100

practice
Y01852    11.700972
Y03006     7.342237
Y03668     6.154320
G81703     5.126073
Y04997     4.963767
            ...    
D81024     1.697803
A81033     1.694036
P81684     1.692085
N81121     1.686724
P81099     1.680349
Length: 100, dtype: float64

In [108]:
practices.loc[practices["code"]=="Y01852"]

,code,name,addr_1,addr_2,borough,village,post_code
7954,Y01852,NATIONAL ENHANCED SERVICE,NYE BEVAN HOUSE,MACLURE ROAD,ROCHDALE,LANCASHIRE,OL11 1DN


In [114]:
results=practices.loc[practices["code"].isin(first_100.index)]

In [113]:
practices["code"].isin(first_100.index).value_counts()

False    11907
True       113
Name: code, dtype: int64

In [129]:
results.shape

(113, 7)

In [133]:
results.drop_duplicates('code').shape

(100, 7)

In [118]:
uniq=results["code"].unique()

In [121]:
anomalies=[]
for u in uniq:
    i=results.loc[results["code"]==u,"name"].sort_values().index[0]
    p_name=results.loc[i,"name"]
    anomalies.append(tuple([u,p_name,opioid_scores[u],scripts_per_practice[u]]))

In [122]:
anomalies[0:2]

[('A81011', 'CHADWICK PRACTICE', 2.2935281254806053, 1569),
 ('A81033', 'OAKFIELD MEDICAL PRACTICE', 1.6940359397112987, 1056)]

In [ ]:
#unique_practices = ...
#anomalies = [("Y01852", "NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100

In [123]:
sorted(anomalies,key=lambda x:x[2],reverse=True)

[('Y01852', 'NATIONAL ENHANCED SERVICE', 11.700971787589962, 7),
 ('Y03006', 'OUTREACH SERVICE NH / RH', 7.342237296165327, 2),
 ('Y03668', 'BRISDOC HEALTHCARE SERVICES OOH', 6.154319628928264, 60),
 ('G81703', 'H&R P C SPECIAL SCHEME', 5.126072754058587, 36),
 ('Y04997', 'HMR BARDOC OOH', 4.963767331557864, 321),
 ('Y04585', 'INTEGRATED CARE 24 LTD (CWSX OOH)', 4.894191128239315, 426),
 ('P81051', 'DARWEN HEALTHCARE', 4.836873272398607, 1919),
 ('P87017', 'THE LIMES MEDICAL PRACTICE', 4.5405213699735345, 1323),
 ('Y01086', 'IC24 LTD (BRIGHTON & HOVE OOH)', 4.339853965026435, 357),
 ('Y05327', 'OLDHAM 7 DAY ACCESS HUB2 OOH', 4.314736414688714, 56),
 ('Y00502', 'IC24 LTD (NORFOLK & WISBECH OOH)', 4.262796706819221, 489),
 ('Y04657', 'ROSSENDALE MIU & OOH', 4.259233601224412, 18),
 ('Y00016', 'BURY WALK-IN CENTRE', 4.154165856095579, 138),
 ('Y04587', 'IC24 LTD (HORSHAM & MID SUSSEX OOH)', 3.7855197840129198, 215),
 ('Y04921', 'LCW HOUNSLOW CCG OOH', 3.5856395010396973, 69),
 ('Y05419', 

In [124]:
grader.score.dw__script_anomalies(anomalies)

Your score: 1.000


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the growth rate in prescription rate from June 2016 to January 2017 for each `bnf_name`. The growth is defined as

$$
\text{growth rate} = \frac{x_1 - x_0}{x_0},
$$
where $x_1$ is the current value and $x_0$ is the previous value.


We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative growth rate) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [135]:
scripts16 = pd.read_csv('./dw-data/201606scripts_sample.csv.gz')

In [136]:
# calculate the growth rate
growth = (scripts.bnf_name.value_counts() - scripts16.bnf_name.value_counts())/scripts16.bnf_name.value_counts()

In [137]:
growth

365 Film 10cm x 12cm VP Adh Film Dress           NaN
365 Film 15cm x 20cm VP Adh Film Dress           NaN
365 Film 4cm x 5cm VP Adh Film Dress             NaN
365 Non Adherent 10cm x 10cm Pfa Plas Fa    0.000000
365 Non Adherent 10cm x 20cm Pfa Plas Fa         NaN
                                              ...   
nSpire PiKo-1 Stnd Range Peak Flow Meter   -0.500000
nSpire Pocket Peak Low Range Peak Flow M    1.000000
nSpire Pocket Peak Stnd Range Peak Flow     1.500000
oraNurse_Toothpaste Orig (1450ppm)          1.000000
palmdoc (Reagent)_Strips                    0.666667
Name: bnf_name, Length: 15424, dtype: float64

In [138]:
# Combine the two series in a Dataframe
merged = pd.DataFrame(dict(growth = growth, item16 = scripts16.bnf_name.value_counts())).reset_index()

In [139]:
merged.fillna(0, inplace=True)

In [140]:
merged

,index,growth,item16
0,365 Film 10cm x 12cm VP Adh Film Dress,0.000000,0.0
1,365 Film 15cm x 20cm VP Adh Film Dress,0.000000,1.0
2,365 Film 4cm x 5cm VP Adh Film Dress,0.000000,1.0
3,365 Non Adherent 10cm x 10cm Pfa Plas Fa,0.000000,3.0
4,365 Non Adherent 10cm x 20cm Pfa Plas Fa,0.000000,0.0
...,...,...,...
15419,nSpire PiKo-1 Stnd Range Peak Flow Meter,-0.500000,2.0
15420,nSpire Pocket Peak Low Range Peak Flow M,1.000000,2.0
15421,nSpire Pocket Peak Stnd Range Peak Flow,1.500000,2.0
15422,oraNurse_Toothpaste Orig (1450ppm),1.000000,2.0


In [142]:
merged = merged[merged['item16'] >= 50]

In [144]:
#Sort by growth rate
merged.sort_values(by='growth', ascending = False, inplace=True)

<ipython-input-144-e23aae8e39f7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.sort_values(by='growth', ascending = False, inplace=True)


In [145]:
script_growth = list(pd.concat([merged.head(50), merged.tail(50)]).itertuples(index=False, name=None))

In [ ]:
script_growth = [("Butec_Transdermal Patch 5mcg\/hr", 3.4677419355, 62.0)] * 100

In [146]:
grader.score.dw__script_growth(script_growth)

Your score: 1.000


## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [ ]:
p = ...
rates = ...
rare_codes = ...
scripts['rare'] = ...

In [148]:
p = 1/scripts.bnf_code.nunique()

rates = scripts['bnf_code'].value_counts() / len(scripts)

mask = rates < .1 * p

rare_codes = rates[mask].index.unique()

scripts['rare'] = scripts['bnf_code'].isin(rare_codes)

Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [ ]:
rare_cost_prop = ...

In [149]:
# rare script act_cost per practice/total act_cost per practice
rare_cost_prop = (scripts[scripts['rare']].groupby('practice')['act_cost'].sum()/scripts.groupby('practice')['act_cost'].sum()).fillna(0)

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [ ]:
relative_rare_cost_prop = ...

In [150]:

relative_rare_cost_prop = (rare_cost_prop - scripts[scripts['rare']]['act_cost'].sum() /scripts['act_cost'].sum())

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [ ]:
standard_errors = ...

In [151]:
standard_error = relative_rare_cost_prop.std()

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(practice_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [ ]:
rare_scores = ...

In [ ]:
rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100

In [ ]:
grader.score.dw__rare_scripts(rare_scripts)

*Copyright &copy; 2021 WorldQuant University. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*